<a href="https://colab.research.google.com/github/rishi-jaizz/Deep-Learning/blob/main/Transfer_Learning_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Library

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

#Load the MNIST dataset

In [3]:
(train_images, train_labels), (test_images,
test_labels) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


#Preprocessing the data: Reshaping and normalizing the images

In [4]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

#Resize images to 32x32 (as VGG16 requires at least 32x32) and keep the last dimension as 1

In [5]:
train_images_resized = tf.image.resize(train_images,(32, 32))
test_images_resized = tf.image.resize(test_images,(32, 32))

#Since VGG16/Mobilenet requires 3-channel input,we replicate the grayscale image across 3 channels

In [6]:
train_images_rgb = tf.image.grayscale_to_rgb(train_images_resized)
test_images_rgb = tf.image.grayscale_to_rgb(test_images_resized)

#Convert labels to categorical format (one-hot encoding)

In [7]:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

#Load a pretrained MobileNetV2 model without the classification layer

In [8]:
base_model = VGG16(weights='imagenet', include_top=False,
input_shape=(32, 32, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


#Freeze the pretrained layers

In [9]:
base_model.trainable = False

#Add custom layers on top of the pre-trained model

In [10]:
model = models.Sequential([
base_model, # Use VGG16 as the feature extractor
layers.Flatten(), # Flatten the output of VGG16 to feed it to the Dense layer
layers.Dense(64, activation='relu'), # Add a Dense layer with 64 units and ReLU activation
layers.Dense(10, activation='softmax') # Output layer for classification into 10 classes (digits)
])

#Compile the model

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Train and Evaluate

In [12]:
# Train the model
history = model.fit(train_images_rgb, train_labels,epochs=5, batch_size=64, validation_split=0.2)
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_rgb,
test_labels)
print(f'Test Accuracy: {test_acc}')

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.7447 - loss: 0.8930 - val_accuracy: 0.9352 - val_loss: 0.2304
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.9369 - loss: 0.2158 - val_accuracy: 0.9482 - val_loss: 0.1758
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9505 - loss: 0.1628 - val_accuracy: 0.9553 - val_loss: 0.1427
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9578 - loss: 0.1364 - val_accuracy: 0.9569 - val_loss: 0.1353
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9643 - loss: 0.1193 - val_accuracy: 0.9588 - val_loss: 0.1276
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9582 - loss: 0.1285
Test Accuracy: 0.9621999859809875
